# Step 2: Private Prediction using Syft Keras (Server)

In [1]:
#import torch
import tensorflow as tf
import numpy as np
import syft
import tf_encrypted as tfe
# hook = syft.TorchHook(torch)
# alice = syft.VirtualWorker(hook, id="alice")  # alice computes on behalf of data owner
# bob = syft.VirtualWorker(hook, id="bob")      # bob computes on behalf of model owner
# carol = syft.VirtualWorker(hook, id="carol")  # carol acts as crypto producer

In [2]:
#import tf_encrypted as tfe
from syft.keras.model import Sequential
from syft.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

In [3]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [4]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_size=1, input_shape=task_shape[1:]))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


In [5]:
# Load pre-trained weights
model.load_weights(pre_trained_weights)

In [6]:
data = np.ones([1, 28, 28, 1])

In [7]:
public_prediction = model.predict(data)
print("Public Prediction:", public_prediction)

Public Prediction: [[  0.33873752  -8.790154     2.502937     3.719169   -10.0552845
   -2.4409113    2.5351093   -9.349673     5.1688886   -5.9522614 ]]


### Secure the model by sharing the weights

In [8]:
prot = tfe.protocol.SecureNN()

tfe_model = model.share(prot=prot)

INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


seq__init__ <tf_encrypted.protocol.pond.pond.Pond object at 0x13436a320>


INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.
INFO:tf_encrypted:Performing an activation before a pooling layer can result in unnecessary performance loss. Check model definition in case of missed optimization.


In [9]:
with prot:
    x = tfe.define_private_variable(data)

    y = tfe_model(x)

In [10]:
with tfe.Session() as sess:
    sess.run(tf.global_variables_initializer())
    actual = sess.run(y.reveal())

INFO:tf_encrypted:Players: ['server0', 'server1', 'server2']


In [11]:
print("Private Prediction: ", actual)

Private Prediction:  [[  0.67885993  -9.56729157   2.93933852   3.85596708 -10.32357872
   -3.16552355   2.83859168 -10.02804451   5.51684194  -6.57872276]]
